#### Pick similar movies using Gemini instead of using cosine search on embeddings

In [13]:
import re
import os
import json
from tqdm import tqdm
from glob import glob
from PIL import Image
from google import genai

from pydantic import BaseModel, TypeAdapter

In [ ]:
with open('src/server/movies_emoji.json') as f:
    list_of_movies = json.load(f)
    print(list_of_movies[0])

154

In [73]:
for movie in list_of_movies:
    del movie['hash']
    del movie['embedding']

In [75]:
apikey = os.environ.get("GEMINI_API_KEY")

In [76]:
client = genai.Client(api_key=apikey)

In [77]:
class Movie(BaseModel):
  provided_movie: str
  fun_fact:str
  similar_movies: list[str]

In [78]:
with open('src/server/movies_emoji.json') as f:
    all_movies = json.load(f)

In [85]:
index = 0
listed = []
for movie in tqdm(list_of_movies):
    response = client.models.generate_content(
    model="gemini-2.0-flash", contents=f"Choose three movies similar to the provided movie: {movie['name']} from the list below. Note, do not pick a sequel or prequel, instead use the given names and descriptions of the movie in order to pick the similar movies. Also return a fun fact about the movie provided.: \n{list_of_movies}",
    config={
        'response_mime_type': 'application/json',
        'response_schema': list[Movie],
    })
    movie_details = response.parsed[0]
    new_movie = all_movies[index]
    new_movie["funfact"] = movie_details.fun_fact
    new_movie["similar"] = movie_details.similar_movies
    index+=1
    listed.append(new_movie)
    with open('enriched.json','w')as outfile:
        json.dump(listed,outfile,ensure_ascii=False)

100%|██████████| 154/154 [07:33<00:00,  2.94s/it]
